# Introduction

In Chapter 3 we devised a signature v2 algorithm.

In this chapter we implement it and use it.

In [8]:
def check(message, condition: bool):
    if condition:
        print("✓ " + message)
    else:
        print("✗ " + message)

# 1 Check NodeSignature

## 1.0 Check Comparer Inequality

In [11]:
from graph_signature_v2 import NodeSignature, compare_signatures

check(
    """{neighborCount:4} is shown before {neighborCount:3}""",
    compare_signatures(
        NodeSignature(label="A", neighborCount=4),
        NodeSignature(label="B", neighborCount=3),
    )
    < 0,
)

check(
    """{finalIndex:3} is shown before {finalIndex:4}""",
    compare_signatures(
        NodeSignature(label="A", finalIndex=3, neighborCount=1),
        NodeSignature(label="B", finalIndex=4, neighborCount=1),
    )
    < 0,
)

check(
    """{cycleDistance:3} is shown before {cycleDistance:4}""",
    compare_signatures(
        NodeSignature(label="A", cycleDistance=3, neighborCount=1),
        NodeSignature(label="B", cycleDistance=4, neighborCount=1),
    )
    < 0,
)

check(
    """{resolutionStep:1} is shown before {resolutionStep:2}""",
    compare_signatures(
        NodeSignature(label="A", finalIndex=0, neighborCount=1, resolutionStep=1),
        NodeSignature(label="B", finalIndex=0, neighborCount=1, resolutionStep=2),
    )
    < 0,
)

check(
    """{inner neighborCount:2} is shown before {inner neighborCount:1}""",
    compare_signatures(
        NodeSignature(
            label="A", neighborCount=1, neighbors=[NodeSignature(neighborCount=2)]
        ),
        NodeSignature(
            label="B", neighborCount=1, neighbors=[NodeSignature(neighborCount=1)]
        ),
    )
    < 0,
)

✓ {neighborCount:4} is shown before {neighborCount:3}
✓ {finalIndex:3} is shown before {finalIndex:4}
✓ {cycleDistance:3} is shown before {cycleDistance:4}
✓ {resolutionStep:1} is shown before {resolutionStep:2}
✓ {inner neighborCount:2} is shown before {inner neighborCount:1}


# 1.1 Check NodeSignature Comparer Equality

In [ ]:
check(
    """{neighborCount:4} is equal to {neighborCount:4}""",
    compare_signatures(
        NodeSignature(label="A", neighborCount=4),
        NodeSignature(label="B", neighborCount=4),
    )
    == 0,
)

check(
    """{finalIndex:3} is equal to {finalIndex:3}""",
    compare_signatures(
        NodeSignature(label="A", finalIndex=3, neighborCount=1),
        NodeSignature(label="B", finalIndex=3, neighborCount=1),
    )
    == 0,
)

check(
    """{cycleDistance:3} is equal to {cycleDistance:3}""",
    compare_signatures(
        NodeSignature(label="A", cycleDistance=3, neighborCount=1),
        NodeSignature(label="B", cycleDistance=3, neighborCount=1),
    )
    == 0,
)

check(
    """{finishStep:1} is equal to {finishStep:1}""",
    compare_signatures(
        NodeSignature(label="A", finalIndex=0, neighborCount=1, finishStep=1),
        NodeSignature(label="B", finalIndex=0, neighborCount=1, finishStep=1),
    )
    == 0,
)

check(
    """{inner neighborCount:2} is equal to {inner neighborCount:2}""",
    compare_signatures(
        NodeSignature(
            label="A", neighborCount=1, neighbors=[NodeSignature(neighborCount=2)]
        ),
        NodeSignature(
            label="B", neighborCount=1, neighbors=[NodeSignature(neighborCount=2)]
        ),
    )
    == 0,
)

✓ {neighborCount:4} is equal to {neighborCount:4}
✓ {finalIndex:3} is equal to {finalIndex:3}
✓ {cycleDistance:3} is equal to {cycleDistance:3}
✓ {finishStep:1} is equal to {finishStep:1}
✓ {inner neighborCount:2} is equal to {inner neighborCount:2}


# 2 Check process_signatures_pass

## 2.1 Ambiguous Signatures

In [ ]:
print("--- Test 1: Ambiguous Signatures ---")

# 1. Setup: Two identical signatures
initial_signatures = [
    NodeSignature(label="A", neighborCount=4),
    NodeSignature(label="B", neighborCount=4),
]

# 2. Execute the function
final_signatures, made_progress = process_signatures_pass(initial_signatures, 1)

# 3. Check the results
check("No progress should be made for identical signatures", not made_progress)

# We can also check that no signature was finalized
check(
    "Node 'A' should remain ambiguous (no finalIndex)",
    final_signatures[0].finalIndex is None,
)
check(
    "Node 'B' should remain ambiguous (no finalIndex)",
    final_signatures[1].finalIndex is None,
)

--- Test 1: Ambiguous Signatures ---
✓ No progress should be made for identical signatures
✓ Node 'A' should remain ambiguous (no finalIndex)
✓ Node 'B' should remain ambiguous (no finalIndex)


In [12]:
print("--- Test 1.1: NodeSignature Simple Initialization (Collapsed State) ---")

# 1. Setup: Define parameters for a simple NodeSignature
label_t1_1 = "A"
neighbor_count_t1_1 = 3

# 2. Execute: Create the NodeSignature object
sig_t1_1 = NodeSignature(label=label_t1_1, neighborCount=neighbor_count_t1_1)

# 3. Check the results: Verify core attributes and default values for optional ones
check(
    f"Test 1.1.1: Label should be '{label_t1_1}'", 
    sig_t1_1.label == label_t1_1
)
check(
    f"Test 1.1.2: neighborCount should be {neighbor_count_t1_1}",
    sig_t1_1.neighborCount == neighbor_count_t1_1
)
check(
    "Test 1.1.3: finalIndex should be None by default",
    sig_t1_1.finalIndex is None
)
check(
    "Test 1.1.4: neighbors list should be None or empty by default", # Depending on your class implementation
    sig_t1_1.neighbors is None or len(sig_t1_1.neighbors) == 0
)
check(
    "Test 1.1.5: cycleDistance should be None by default",
    sig_t1_1.cycleDistance is None
)
check(
    "Test 1.1.6: expandedStep should be None by default",
    sig_t1_1.resolutionStep is None
)

--- Test 1.1: NodeSignature Simple Initialization (Collapsed State) ---
✓ Test 1.1.1: Label should be 'A'
✓ Test 1.1.2: neighborCount should be 3
✓ Test 1.1.3: finalIndex should be None by default
✓ Test 1.1.4: neighbors list should be None or empty by default
✓ Test 1.1.5: cycleDistance should be None by default
✓ Test 1.1.6: expandedStep should be None by default
